In [ ]:
import os

In [2]:
from keras.preprocessing import image

In [3]:
import cv2

In [4]:
categories = ['with_mask','without_mask']

In [5]:
data = []
for category in categories:
    path = os.path.join('train',category)

    label = categories.index(category)
    
    for file in os.listdir(path):
        
        img_path = os.path.join(path,file)
        img = cv2.imread(img_path)
        img = cv2.resize(img,(224,224))

        data.append([img,label])

        

In [6]:
len(data)

1376

In [7]:
import random

In [8]:
random.shuffle(data)

In [9]:
X = []
y = []
for features,label in data:
    X.append(features)
    y.append(label)

In [10]:
len(X)

1376

In [11]:
len(y)

1376

In [12]:
import numpy as np

In [13]:
X = np.array(X) 
y = np.array(y)

In [14]:
X.shape

(1376, 224, 224, 3)

In [15]:
y.shape

(1376,)

In [16]:
y

array([0, 0, 1, ..., 1, 0, 1])

In [17]:
X = X/255

In [18]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [19]:
X_train.shape

(1100, 224, 224, 3)

In [20]:
X_test.shape

(276, 224, 224, 3)

In [21]:
from keras.applications.vgg16 import VGG16

In [22]:
vgg = VGG16()

In [23]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [24]:
from keras import Sequential

In [25]:
model = Sequential()

In [26]:
for layer in vgg.layers[:-1]:
    model.add(layer)

In [27]:
for layer in model.layers:
    layer.trainable=False

In [28]:
from keras.layers import Dense

In [29]:
model.add(Dense(1,activation='sigmoid'))

In [30]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [31]:
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

In [32]:
H = model.fit(X_train,y_train,epochs=5, validation_data=(X_test,y_test))

Epoch 1/5
35/35 [==============================] - 275s 8s/step - loss: 0.5862 - accuracy: 0.7318 - val_loss: 0.3957 - val_accuracy: 0.9167
Epoch 2/5
35/35 [==============================] - 242s 7s/step - loss: 0.3428 - accuracy: 0.9127 - val_loss: 0.2723 - val_accuracy: 0.9312
Epoch 3/5
35/35 [==============================] - 202s 6s/step - loss: 0.2515 - accuracy: 0.9255 - val_loss: 0.2163 - val_accuracy: 0.9457
Epoch 4/5
35/35 [==============================] - 208s 6s/step - loss: 0.2047 - accuracy: 0.9400 - val_loss: 0.1828 - val_accuracy: 0.9638
Epoch 5/5
35/35 [==============================] - 212s 6s/step - loss: 0.1814 - accuracy: 0.9500 - val_loss: 0.1571 - val_accuracy: 0.9638


In [33]:
cap = cv2.VideoCapture(0)

In [39]:
while True:
    ret, frame = cap.read()
    img = cv2.resize(frame,(224,224))
    y_pred = detect_face_mask(img)

    coods = detect_face(cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY))

    for x,y,w,h in coods:
        cv2. rectangle(frame,(x,y),(x+w,y+h),(255,0,0),3)
        
    if y_pred ==0:
        
        draw_label(frame,"mask found",(30,30),(0,255,0))
    else:
        draw_label(frame,"No mask",(30,30),(0,0,255))
    
    cv2.imshow("window", frame)
    if cv2.waitKey(1) & 0xFF == ord('x'):
        break
cv2.destroyAllWindows()

1/1 [==============================] - 1s 1s/step


In [35]:
def detect_face_mask(img):
    y_pred = (model.predict(img.reshape(1,224,224,3))> 0.5).astype("int32")
    return y_pred[0][0]

In [ ]:
sample1 = cv2.imread('sample2.jpg')
sample1 = cv2.resize(sample1,(224,224))

In [ ]:
detect_face_mask(sample1)

In [36]:
def draw_label(img,text,pos,bg_color):
    text_size = cv2.getTextSize(text,cv2.FONT_HERSHEY_SIMPLEX,1,cv2.FILLED)
    end_x = pos[0] + text_size[0][0] + 2
    end_y = pos[1] + text_size[0][1] + 2

    cv2.rectangle(img,pos,(end_x,end_y),bg_color,cv2.FILLED)
    cv2.putText(img,text,pos,cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),1,cv2.LINE_AA)

In [37]:
haar = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [38]:
def detect_face(img):
    coods = haar.detectMultiScale(img)
    return coods

In [ ]:
# visualize feature maps output from each block in the vgg model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from keras.models import Model
import matplotlib.pyplot as plt
from numpy import expand_dims




# load the model
model = VGG16()
# redefine model to output right after the first hidden layer
ixs = [2, 5, 9, 13, 17]
outputs = [model.layers[i].output for i in ixs]
model = Model(inputs=model.inputs, outputs=outputs)
# load the image with the required shape
# convert the image to an array
img = load_img(f'sample2.jpg', target_size=(224, 224))
# convert the image to an array
img = img_to_array(img)
# expand dimensions so that it represents a single 'sample'
img = expand_dims(img, axis=0)
# prepare the image (e.g. scale pixel values for the vgg)
img = preprocess_input(img)
# get feature map for first hidden layer
feature_maps = model.predict(img)
# plot the output from each block
square = 8
for fmap in feature_maps:
	# plot all 64 maps in an 8x8 squares
	ix = 1
	for _ in range(square):
		plt.figure(figsize=(64,64))
		for _ in range(square):
           

			# specify subplot and turn of axis
			ax = plt.subplot(square, square, ix)
			ax.set_xticks([])
			ax.set_yticks([])
			
			# plot filter channel in grayscale
			plt.imshow(fmap[0, :, :, ix-1], cmap='viridis')
			ix += 1
	# show the figure

        
	plt.show()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# N = 5
# plt.style.use("ggplot")
# ax = plt.gca()
# ax.set_facecolor('white')
# plt.figure()
# plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
# plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
# plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
# plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
# plt.title("Training Loss and Accuracy")
# plt.xlabel("Epoch")
# plt.ylabel("Loss/Accuracy")
# plt.legend(loc="lower left")
# plt.show()
# plt.savefig("plot.png")

In [ ]:
N = 5

# plt.style.use("ggplot")
plt.figure(0)
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend(loc="lower left")
ax = plt.gca()
ax.set_facecolor('white')
plt.grid(color='black', linestyle='-', linewidth=0.2)
#plt.savefig("plot1.png")


In [ ]:
ax = plt.gca()
ax.set_facecolor('white')
plt.figure(1)
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.title('Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend(loc="lower left")
plt.grid(color='black', linestyle='-', linewidth=0.2)
#plt.savefig("plot2.png")